# Multi-height wind data

In this example we explore the calculation of multi-height wind data, as for example obtained from WRF results or downloaded from the [NEWA website](https://map.neweuropeanwindatlas.eu/) at a single point.

Here we will use the static data file `WRF-Timeseries-4464.csv.gz` that is part of the `foxes` static data. It has the following data structure:

```
Time,WS-50,WS-75,...,WS-500,WD-50,WD-75,...,WD-500,TKE-50,TKE-75,...,TKE-500,RHO
2009-01-01 00:00:00,7.37214,7.42685,...,1.28838
...
2009-01-31 23:50:00,10.27767,10.36368,...,1.30095
```

The time stamp column marks one month in 10 minute steps, and the wind speed (WS), wind direction (WD) and turbulent kinetic energy (TKE) are provided at 8 heights between 50 and 500 m. The air density (RHO) does not have height dependency but varies with time.

The basic assumption of this example is that we can calculate our wind farm results based on this data, i.e., that the horizontal variation can be neglected (for completely heterogeneous inflow data, see the corresponding example).

These are the imports for this example:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from xarray import open_dataset

import foxes
import foxes.variables as FV
import foxes.constants as FC

First, we setup the model book and the wind farm. We choose 5 turbines in a row:

In [ ]:
# create wind farm, a single row of turbines:
farm = foxes.WindFarm()
foxes.input.farm_layout.add_row(
    farm=farm,
    xy_base=[0.0, 0.0],
    xy_step=[600.0, 0.0],
    n_turbines=5,
    turbine_models=["NREL5MW"],
    H=200.,
    verbosity=0,
)

ax = foxes.output.FarmLayoutOutput(farm).get_figure(figsize=(5,3))
plt.show()

Note that we manually change the hub height from 90 m to 200 m here. Next, we create the states based on the static data file `WRF-Timeseries-3000.nc`:

In [ ]:
# This is what the nc file looks like:
fpath = foxes.StaticData().get_file_path(foxes.STATES, "WRF-Timeseries-3000.nc")
open_dataset(fpath)

Now let's create the corresponding states object:

In [ ]:
states = foxes.input.states.MultiHeightNCTimeseries(
    data_source="WRF-Timeseries-3000.nc",
    time_coord="Time",
    h_coord="height",
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2col={FV.WS: "ws", FV.WD: "wd", FV.TI: "ti", FV.RHO: "rho"},
)

o = foxes.output.StatesRosePlotOutput(states, point=[0., 0., 100.])
fig = o.get_figure(16, FV.AMB_WS, [0, 3.5, 6, 10, 15, 20], figsize=(6, 6))
plt.show()

Note how the `var2col` option offers a mapping from the expected to the actual column names, if needed.

Let's next setup our algorithm. Notice that we include the z-sensitive rotor model `level10`, with 10 points on a vertical line (also the `grid` models would be an option). The partial wakes choice `None` represents default settings for all wake models. It is important that we do not select `rotor_points` together with the `level10` rotor, since averaging over a vertical line of points does not make much sense.

In [ ]:
algo = foxes.algorithms.Downwind(
    farm,
    states,
    rotor_model="level10",
    wake_models=["Bastankhah2014_linear_ka02"],
    partial_wakes=None,
    verbosity=0,
)

We now visualize the vertical flow profile. Alternatively to the here shown use of the `states_sel` parameter in the plotting function, we could have reduced the whole `states` object by `states.reset(states_loc=["2009-01-06 13:50:00"])`.

In [ ]:
farm_results = algo.calc_farm()

o = foxes.output.FlowPlots2D(algo, farm_results)
g = o.gen_states_fig_xz(FV.AMB_WS, resolution=10, x_direction=270,
        xmin=0., xmax=1000., zmin=50., zmax=500., figsize=(8,6), states_sel=["2009-01-06 13:50:00"])
fig = next(g)
plt.show()

We can now calculate the full states results:

In [ ]:
farm_results = algo.calc_farm()

fr = farm_results.to_dataframe()
print(fr[[FV.WD, FV.REWS, FV.P]])

o = foxes.output.FarmLayoutOutput(farm, farm_results)
o.get_figure(color_by="mean_REWS", title="Mean REWS [m/s]", s=150, annotate=0)
plt.show()

o = foxes.output.FarmResultsEval(farm_results)
P0 = o.calc_mean_farm_power(ambient=True)
P = o.calc_mean_farm_power()
print(f"\nFarm power        : {P/1000:.1f} MW")
print(f"Farm ambient power: {P0/1000:.1f} MW")
print(f"Farm efficiency   : {o.calc_farm_efficiency()*100:.2f} %")
print(f"Annual farm yield : {o.calc_farm_yield(algo=algo):.2f} GWh")